In [1]:
import os
import re
import pandas as pd
import numpy as np
#import nltk
import spacy
from spacy.matcher import Matcher
#from nltk.corpus import stopwords
nlp=spacy.load('en_core_web_sm')
matcher=Matcher(nlp.vocab)
import PyPDF2
from pywintypes import com_error
import win32com.client as win32

In [3]:
def extract_names(document):
    nlp_text=nlp(document)
    pattern=[{'POS':'PROPN'},{'POS':'PROPN'}]
    matcher.add('NAME',None,pattern)
    matches=matcher(nlp_text)
    for match_id,start,end in matches:
        span=nlp_text[start:end]
        return span.text

In [4]:
def extract_email_addresses(text):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(text)

In [6]:
def extract_mobile_number(text):
    #mno = re.findall(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,15}[0-9]', text)
    mno = re.findall(r'[\+\(]?[1-9][0-9 \-\(\)]{8,15}[0-9]', text)
    mono = []
    for i in range(len(mno)):
        digit = 0
        for j in mno[i]:
            if j.isnumeric():
                digit+=1
        if digit > 9 and digit < 15:
            mono.append(mno[i])         
    return mono

In [8]:
def extract_skill_set(text):
    with open("C:\\Users\\PritamDevadattaJena\\Desktop\\alisha\\skills.csv","r") as skill:
        skill_set = skill.read().split("\n")    
    f = []
    for s in skill_set:
        #if re.search(s, text, re.I):
        if s in text:
            if len(s)>=1:
                f.append(s)
    
    return f

In [10]:
def generate_ngrams(filename, n):
    
    words = filename.split()
    output = []  
    for i in range(len(words)-n+1):
        output.append(words[i:i+n])
    f=[]            
    for i in output:
        if 'years' in i:
            f.append(output[output.index(i)])
            if len(f)==1:
                n=f[0][0]
                n=n + " " + "years"
                break
    
    if len(f)<1:
        n='Not specified'
    return n

In [12]:
def exper(fullText):
    mi=fullText.lower()
    #print(mi)
    h=mi.replace("_"," ")
    h=h.replace("-"," ")
    h=h.replace(","," ")
    h=h.replace("("," ")
    h=h.replace(")"," ")
    h=h.replace(".docx"," ")
    h=h.replace(".pdf"," ")
    h=h.split()              #look at h only years get it
    if 'years' in h and 'months' in h:
        d=h[h.index('years')-1] + " " + h[h.index('years')]+ " " +h[h.index('months')-1] + " " +h[h.index('months')]
    #elif 'year' in h:
        #d=h[h.index('year')-1] + " " + h[h.index('year')]
    elif 'years' in h:
        d=h[h.index('years')-1] + " " + h[h.index('years')]
    elif 'months' in h:
        d=h[h.index('months')-1] + " " + h[h.index('months')]
    #elif 'month' in h:
     #   d=h[h.index('month')-1] + " " + h[h.index('month')]
    elif re.search('no experience',str(h),re.M|re.I) :
        d='No Experience'
    else:
        d=generate_ngrams(fullText, 2)  
    return d   

In [22]:
####IMPORTING DOC & DOCX FILES ONLY
def full_resume():
    path = "C:\\Users\\LENOVO\\Desktop\\res\\resume\\"
    data = pd.DataFrame(columns = ["FileName","FileContents","Name","Email Address","Contact Number","Experience"])
    word = win32.Dispatch("Word.Application")
    word.Visible = 0
    try:
        for filename in os.listdir(path):
            if filename.endswith(('.doc','.docx')):
                print(filename)
                DOC_FILEPATH = path + filename        
                doc = win32.GetObject(DOC_FILEPATH)
                res = doc.Range().Text           
            name=extract_names(res)
            email = extract_email_addresses(res)
            cno = extract_mobile_number(res)
            #skills = extract_skill_set(res)
            exp= exper(res)
            #python_count, excel_count,vba_count = count_skills(res)
            data = data.append({"FileName":filename, "FileContents":res, "Name":name, "Email Address":email,\
                    "Contact Number":cno,"Experience": exp},ignore_index=True)        
            #df1 = pd.DataFrame(data, columns = ["FileName","FileContents","Name","Email Address","Contact Number","Experience"])    
            #df.to_csv("resumes_filter8.csv", index=False)
            #doc.Saved = False
            #doc.Save()
            #doc.Close()
        word.Quit()
    except com_error:
        print('error')
        pass

    ####IMPORTING PDF ONLY
    for filename in os.listdir(path):
        if filename.endswith('.pdf'):
            print(filename)
            pdfFileObj = open(path + filename ,'rb')     #'rb' for read binary mode
            pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
            number_of_pages = pdfReader.getNumPages()
            ls=[]
            for page_number in range(number_of_pages):   # use xrange in Py2
                page = pdfReader.getPage(page_number)
                page_content = page.extractText()
                ls.append(page_content)
                res = ''.join(''.join(map(str, row)) for row in ls)    
            name=extract_names(res)
            email = extract_email_addresses(res)
            cno = extract_mobile_number(res)
            #skills = extract_skill_set(res)
            exp= exper(res)
            #python_count, excel_count,vba_count = count_skills(res)
            data = data.append({"FileName":filename, "FileContents":res, "Name":name, "Email Address":email,\
                    "Contact Number":cno,"Experience": exp},ignore_index=True)
            
            #df2 = pd.DataFrame(data, columns = ["FileName","FileContents","Name","Email Address","Contact Number","Experience"])    
            #df.to_csv("resumes_filter8.csv", index=False)
    return data

In [23]:
full_resume()

Abhijeet_Resume_DA (4).docx
Asha_Resume.doc
Himanshu_CV.docx
Jagruti J Resume.doc
Santhosh Kumar S P.docx
Shiva Kotturi resume.docx
~$ha_Resume.doc
error
Ashis_3yr_Infosys.pdf
Ayan_cv_pdf.pdf
Copy of Pooja Resume Data_-2019.pdf
DilipBehera_Resume.pdf
Kuntal RoyChowdhury_Resume.pdf
Megha_Resume.pdf
Shashikanth Kadam - Data Science   New.pdf


,FileName,FileContents,Name,Email Address,Contact Number,Experience
0,Abhijeet_Resume_DA (4).docx,ABHIJEET GUPTA\t\t\t +919591508918\t\t\t ...,ABHIJEET GUPTA,"[abhi.jeet_gupta@yahoo.com, abhi.jeet_gupta@ya...","[+919591508918, +91 9591508918]",4.5 years
1,Asha_Resume.doc,Asha H R\rMobile: +91 9538044096\rE-Mail: asha...,E-,[ashahr.jc@gmail.com],[+91 9538044096],3.2 years
2,Ashis_3yr_Infosys.pdf,Asha H R\rMobile: +91 9538044096\rE-Mail: asha...,E-,[ashahr.jc@gmail.com],[+91 9538044096],3.2 years
3,Ayan_cv_pdf.pdf,Asha H R\rMobile: +91 9538044096\rE-Mail: asha...,E-,[ashahr.jc@gmail.com],[+91 9538044096],3.2 years
4,Copy of Pooja Resume Data_-2019.pdf,Asha H R\rMobile: +91 9538044096\rE-Mail: asha...,E-,[ashahr.jc@gmail.com],[+91 9538044096],3.2 years
5,DilipBehera_Resume.pdf,Asha H R\rMobile: +91 9538044096\rE-Mail: asha...,E-,[ashahr.jc@gmail.com],[+91 9538044096],3.2 years
6,Himanshu_CV.docx,Himanshu Goyal Chandigarh +91-8591028324 himan...,Himanshu Goyal,[himanshu.4991@gmail.com],[+91-8591028324],6.5 years
7,Jagruti J Resume.doc,Jagruti Khatri\rSr. Business Analyst– Tata Con...,Jagruti Khatri,[jagrutikhatri20@gmail.com],[91-9033935248],Not specified
8,Kuntal RoyChowdhury_Resume.pdf,Jagruti Khatri\rSr. Business Analyst– Tata Con...,Jagruti Khatri,[jagrutikhatri20@gmail.com],[91-9033935248],Not specified
9,Megha_Resume.pdf,Jagruti Khatri\rSr. Business Analyst– Tata Con...,Jagruti Khatri,[jagrutikhatri20@gmail.com],[91-9033935248],Not specified
